In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
network_pairs = ['1_2','1_3','1_4','1_5','1_6','1_7','1_8','1_9','1_10','1_11','1_12','1_13','1_14','1_15','1_16','1_17',
'2_3','2_4','2_5','2_6','2_7','2_8','2_9','2_10','2_11','2_12','2_13','2_14','2_15','2_16','2_17',
'3_4','3_5','3_6','3_7','3_8','3_9','3_10','3_11','3_12','3_13','3_14','3_15','3_16','3_17',
'4_5','4_6','4_7','4_8','4_9','4_10','4_11','4_12','4_13','4_14','4_15','4_16','4_17',
'5_6','5_7','5_8','5_9','5_10','5_11','5_12','5_13','5_14','5_15','5_16','5_17',
'6_7','6_8','6_9','6_10','6_11','6_12','6_13','6_14','6_15','6_16','6_17',
'7_8','7_9','7_10','7_11','7_12','7_13','7_14','7_15','7_16','7_17',
'8_9','8_10','8_11','8_12','8_13','8_14','8_15','8_16','8_17',
'9_10','9_11','9_12','9_13','9_14','9_15','9_16','9_17',
'10_11','10_12','10_13','10_14','10_15','10_16','10_17',
'11_12','11_13','11_14','11_15','11_16','11_17',
'12_13','12_14','12_15','12_16','12_17',
'13_14','13_15','13_16','13_17',
'14_15','14_16','14_17',
'15_16','15-17',
'16_17']

## Vectorize Connectomes Example (single run)

In [5]:
subs_file = pd.read_csv('/path_to_connectomes/yeo17/cobre/cobre.csv')

In [6]:
conn_mtx = []
subs = subs_file['subjects']
for i in range(0,len(subs_file)):
    SR = subs_file['subjects'].iloc[i]
    conn = np.array(pd.read_csv(f'/path_to_connectomes/yeo17/cobre/{SR}.txt',
                      delim_whitespace=True,header=None).iloc[0:17,0:17])
    conn_mtx.append(conn)

In [7]:
### vectorize the matrices
utri_idxs = np.triu_indices(17, k=1)
good_edges = np.zeros((len(conn_mtx),len(utri_idxs[0])))

for i in range(0,len(conn_mtx)):
    good_edges[i,:] = conn_mtx[i][utri_idxs]
print(good_edges.shape)
good_edges = np.arctanh(good_edges)  ## fisher z-transform

(148, 136)


In [8]:
cobre = pd.DataFrame(data=good_edges, columns=network_pairs)

In [9]:
cobre['sub_id'] = subs

In [10]:
cobre.to_csv('/path_to_connectomes/yeo17/cobre/cobre_rest_vectorized.csv', index=False)

## Vectorize Connectomes Example (multiple runs)

In [105]:
subs_file = pd.read_csv('/path_to_connectomes/yeo17/HCP_Aging/hcpage.csv')

In [106]:
subs_file.dropna(inplace=True)

In [107]:
conn_mtx = []
avgs = []
subs = []
runs = 0
for i in range(0,len(subs_file)):
    sub = subs_file['subs'].iloc[i]
    runs = int(subs_file['runs'].str.replace('run-','').iloc[i])
    SR = subs_file['sub_runs'].iloc[i]
    conn = np.array(pd.read_csv(f'/path_to_connectomes/yeo17/HCP_Aging/{SR}.txt',
                      delim_whitespace=True,header=None).iloc[0:17,0:17])
    conn_mtx.append(conn)
    assert SR == f'{sub}_run-{runs}'
    if (sub == subs_file['subs'].iloc[i-1]) & (i+2 > len(subs_file)):
        subs.append(sub)
        for k in range(0,runs):
            new_mtx = conn_mtx[i-1*k]
            if k == 0:
                sum_mtx = new_mtx
            else:
                sum_mtx = [[sum_mtx[i][j] + new_mtx[i][j] for j in range(len(sum_mtx[0]))] for i in range(len(sum_mtx))]
        avg = np.divide(sum_mtx,runs)
        avgs.append(avg)
    elif (sub == subs_file['subs'].iloc[i-1]) & (sub != subs_file['subs'].iloc[i+1]):
        subs.append(sub)
        for k in range(0,runs):
            new_mtx = conn_mtx[i-1*k]
            if k == 0:
                sum_mtx = new_mtx
            else:
                sum_mtx = [[sum_mtx[i][j] + new_mtx[i][j] for j in range(len(sum_mtx[0]))] for i in range(len(sum_mtx))]
        avg = np.divide(sum_mtx,runs)
        avgs.append(avg)

In [108]:
### vectorize the matrices
utri_idxs = np.triu_indices(17, k=1)
good_edges = np.zeros((len(avgs),len(utri_idxs[0])))

for i in range(0,len(avgs)):
    good_edges[i,:] = avgs[i][utri_idxs]
print(good_edges.shape)
good_edges = np.arctanh(good_edges)  ## fisher z-transform

(725, 136)


In [109]:
hcpage = pd.DataFrame(data=good_edges, columns=network_pairs)

In [110]:
len(subs)

725

In [111]:
hcpage['sub_id'] = subs

In [112]:
hcpage.to_csv('/path_to_connectomes/yeo17/HCP_Aging/hcpage_rest_vectorized.csv', index=False)